# Chapter 14 codes

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# "First, let’s implement a very simple RNN model, without using 
# any of TensorFlow’s RNN operations, to better understand what goes on under the hood."


n_inputs = 3
n_neurons = 5

# This is a single cell (layer) RNN with 5 neurons in the cell

# Takes an input vector of size n_inputs at each step

# The RNN only runs through 2 time steps

X0 = tf.placeholder(tf.float32, [None, n_inputs])
print(X0)

X1 = tf.placeholder(tf.float32, [None, n_inputs])
print(X1)

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
print(Wx)
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
print(Wy)
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))
print(b)

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
print(Y0)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)
print(Y1)

init = tf.global_variables_initializer()

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)
<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(5, 5) dtype=float32_ref>
<tf.Variable 'Variable_2:0' shape=(1, 5) dtype=float32_ref>
Tensor("Tanh:0", shape=(?, 5), dtype=float32)
Tensor("Tanh_1:0", shape=(?, 5), dtype=float32)


In [3]:
import numpy as np

# Mini-batch:        instance 0,instance 1,instance 2,instance 3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [4]:
print(Y0_val) # Output at t = 0
print()
print(Y1_val) # Output at t = 1


# There are 4 rows of outputs (1 for each fed instance)
# There are 5 columns: 1 for each output neuron (see n_neurons)

[[ 0.99901783  0.9828505  -0.84773266  0.9465726  -0.5290006 ]
 [ 1.          1.         -0.9994184   1.         -0.9991877 ]
 [ 1.          1.         -0.99999815  1.         -0.9999989 ]
 [ 1.          1.          0.8593841   1.         -1.        ]]

[[ 1.          1.         -0.93551964  1.         -1.        ]
 [ 0.36935315 -0.9849068   0.9999947   0.86651397 -1.        ]
 [ 1.          1.          0.96580434  1.         -1.        ]
 [ 0.99997663  0.999373   -0.01769763  1.         -0.9999705 ]]


In [ ]:
# Creating the same model using TF

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])


# Creating the cell (layer)
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

# static_rnn creates an unrolled network by chaining cells
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1],
                                                dtype=tf.float32)
Y0, Y1 = output_seqs